In [2]:
import tensorflow as tf
import tensorflow_datasets
import transformers
# from transformers import *

print(tf.__version__)
print(transformers.__version__)

2.0.0-rc1
2.2.2


In [6]:
MODEL_NAME='hfl/chinese-bert-wwm'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = transformers.BertModel.from_pretrained(MODEL_NAME)


A Jupyter Widget

In [11]:
import pprint
pprint.pprint(dir(model))
pprint.pprint(type(model))
pprint.pprint(type(model.get_input_embeddings()))




['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backend',
 '_backward_hooks',
 '_buffers',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_get_resized_embeddings',
 '_init_weights',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_parameters',
 '_prune_heads',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_resize_token_embeddings',
 '_slow_forward',
 '_state_dict_hooks',
 '_tie_or_clone_weights',
 '_tracing_name',
 '_version',
 'add_module',
 'apply',
 'base_model',
 'base_model_prefix',
 'buffers',
 'children',
 'config',
 'config_c

In [ ]:
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='mrpc')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task='mrpc')
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
valid_dataset = valid_dataset.batch(64)
print(type(valid_dataset))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

history = model.fit(train_dataset, epochs=2, steps_per_epoch=115,
                    validation_data=valid_dataset, validation_steps=7)

print(history)

In [ ]:
print(history.history)

In [ ]:
model.save_pretrained('./save/')

In [12]:
pytorch_model = BertForSequenceClassification.from_pretrained('./save/', from_tf=True)

In [15]:
sentence_0 = "This research was consistent with his findings."

sentence_1 = "His findings were compatible with this research."
sentence_2 = "His findings were not compatible with this research."

inputs_1 = tokenizer.encode_plus(sentence_0, sentence_1, add_special_tokens=True, return_tensors='pt')
inputs_2 = tokenizer.encode_plus(sentence_0, sentence_2, add_special_tokens=True, return_tensors='pt')

print(inputs_1)

# pred_1 = pytorch_model(*inputs_1)[0].argmax().item()
# pred_2 = pytorch_model(*inputs_2)[0].argmax().item()

# print("sentence_1 is", "a paraphrase" if pred_1 else "not a paraphrase", "of sentence_0")
# print("sentence_2 is", "a paraphrase" if pred_2 else "not a paraphrase", "of sentence_0")

{'input_ids': tensor([[  101,  8554, 10018,  9947, 11485, 10500, 11598,  8165,  8663, 10163,
         11707, 10334,   119,   102, 10163, 11707, 10334,  8997,  8358,  8134,
          8797,  9007,  8862,  8663,  8554, 10018,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}
